## Actividad 2 NLP

## Equipo:  
* Diego Arturo Padilla Domínguez - A01552594
* Keyuan Zhao - A01366831
* Carolina Herrera Martínez - A01411547
* Cutberto Arizabalo Nava - A01411431
* Jose Pablo Cobos Austria - A01274631



In [1]:
pip uninstall torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [3]:
!pip install torchtext

In [4]:
!pip install torchdata

In [3]:
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.nn import functional as F
# Dataset and dataloader
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

In [4]:
a=torch.cuda.FloatTensor()

In [66]:
torch.cuda.is_available()

True

In [67]:
# Use Gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
train_dataset,  test_dataset = AG_NEWS()

In [8]:
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [9]:
len(train_dataset), len(test_dataset)

(120000, 7600)

In [10]:
train_dataset[:10]

[(3,
  "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."),
 (3,
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'),
 (3,
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums."),
 (3,
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.'),
 (3,
  'Oil prices soar to all-time record, 

In [11]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
  for _, text in data:
    yield tokeniser(text)

In [12]:
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [13]:
len(vocab)

95811

In [14]:
for i in range(20):
  print(i, vocab.lookup_token(i))

0 <unk>
1 .
2 the
3 ,
4 to
5 a
6 of
7 in
8 and
9 s
10 on
11 for
12 #39
13 (
14 )
15 -
16 '
17 that
18 with
19 as


In [15]:
tokens = tokeniser('Welcome to TE3007B')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007b'] [3314, 4, 0]


In [16]:
NUM_TRAIN = int(len(train_dataset) * 0.9)
NUM_VAL =len(train_dataset) - NUM_TRAIN

In [17]:
NUM_VAL

12000

In [18]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [19]:
len(train_dataset), len(val_dataset), len(test_dataset)

(108000, 12000, 7600)

In [20]:
max_tokens = 50

In [21]:
def collate_batch(batch):
  y, x = list(zip(*batch))
  #create list with tokens
  x = [vocab(tokeniser(text)) for text in x]
  # padding or clipping
  x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]
  return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)-1

In [22]:
labels = ['World', 'Sports', 'Business', 'Sci/Tech']
BATCH_SIZE = 1024

In [23]:
# DataLoaders
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
val_loader = DataLoader(val_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
test_loader = DataLoader(test_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)

In [24]:
for i, (x, y) in enumerate(test_loader):
  print(i, x.shape, y.shape)

0 torch.Size([1024, 50]) torch.Size([1024])
1 torch.Size([1024, 50]) torch.Size([1024])
2 torch.Size([1024, 50]) torch.Size([1024])
3 torch.Size([1024, 50]) torch.Size([1024])
4 torch.Size([1024, 50]) torch.Size([1024])
5 torch.Size([1024, 50]) torch.Size([1024])
6 torch.Size([1024, 50]) torch.Size([1024])
7 torch.Size([432, 50]) torch.Size([432])


In [25]:
#let us build our RNN

In [108]:
EMBEDDING_SIZE = 300
NEURONS = 100 #hidden
LAYERS = 4
NUM_CLASSES = 4

In [109]:
class LSTM_Model_1(nn.Module):
  def __init__(self, embed_size, hidden, layers, num_classes):
    super().__init__()
    self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                        embedding_dim=embed_size)
    
    self.lstm = nn.LSTM(input_size=embed_size,
                      hidden_size = hidden,
                      num_layers = layers,
                      batch_first = True)

    self.fc = nn.Linear(in_features=hidden, out_features= num_classes)

  def forward(self, x):
    vector_embs = self.embedding_layer(x)
    y, h = self.lstm(vector_embs)
    return self.fc(y[:,-1])
    

In [110]:
lstm_model = LSTM_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)

In [111]:
lstm_model

LSTM_Model_1(
  (embedding_layer): Embedding(95811, 300)
  (lstm): LSTM(300, 100, num_layers=4, batch_first=True)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)

In [112]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype = torch.long)
            y = y.to(device=device, dtype = torch.long)
            scores = model(x)
            _, preds = scores.max(dim=1)
            num_correct += (preds == y).sum()
            num_total += preds.size(0)
        acc = float(num_correct)/num_total
        return acc

In [113]:
def train(model, optimiser, epochs=100):
  model = model.to(device=device)
  for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
      model.train()
      x = x.to(device=device, dtype=torch.long)
      y = y.to(device=device, dtype=torch.long)
      # run model
      scores = model(x)
      # compute cost
      cost = F.cross_entropy(input=scores, target=y)
      # reset gradient
      optimiser.zero_grad()
      # compute gradient
      cost.backward()
      # update parameter
      optimiser.step()
    acc = accuracy(model, val_loader)
    print(f'Epoch {epoch}, costo {cost.item():.4f}, val acc {acc:.4f}')


In [114]:
epochs = 10
lr = 0.01

lstm_model = LSTM_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(lstm_model.parameters(), lr=lr)

In [115]:
train(lstm_model, optimiser, epochs)

Epoch 0, costo 0.4023, val acc 0.8481
Epoch 1, costo 0.2681, val acc 0.8974
Epoch 2, costo 0.2297, val acc 0.9065
Epoch 3, costo 0.1853, val acc 0.9079
Epoch 4, costo 0.1144, val acc 0.9100
Epoch 5, costo 0.1200, val acc 0.9049
Epoch 6, costo 0.0706, val acc 0.9099
Epoch 7, costo 0.0966, val acc 0.9037
Epoch 8, costo 0.0886, val acc 0.8998
Epoch 9, costo 0.0580, val acc 0.9067


In [117]:
print(f'{accuracy(lstm_model, test_loader):.4f}')

0.9033
